In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Scrapper for "Izquierda Diario"
It can work simply based on bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.laizquierdadiario.com/Economia/"

def parse_url(url):
    response = requests.get(url)
    content = response.content
    parsed_response = BeautifulSoup(content, "html.parser")
    return parsed_response

soup = parse_url(base_url)
here = soup.find('strong', {'class':"on"})

flyer = []
headline = []
url = []

date = []
lead = []
body = []

i = 0
n_news = 0

# arbitrary stop at 300 iterations
while int(here.text) < 600:
    here = soup.find('strong', {'class':"on"})
    print(f"{i + 1}º page, articles from {int(here.text)} to {soup.find_all('a', {'class':'lien_pagination'})[i].text}")

    # strict selection
    news = soup.select("div[class=noticia]")
    n_news += len(news)
    print('News collected:', n_news)

    # parse while reading page source
    for p_news in news:
        link = base_url + p_news.find('a')['href']

        url.append(link)
        if p_news.find('h4') is not None:
            flyer.append(p_news.find('h4').text)
        else:
            flyer.append('')
        headline.append(p_news.find('a').text)

        web = requests.get(link)
        article = BeautifulSoup(web.content, 'html.parser')
        
        if article.find('div', {'class': 'header-articulo'}) is not None:
            if article.find('div', {'class': 'header-articulo'}).find('p') is not None:        
                lead.append(article.find('div', {'class': 'header-articulo'}).find('p').text)
            else:
                lead.append('')
            date.append(article.find('span', {'style': 'color: #364b67;'}).text)

            texto = str()
            for paragraph in article.find('div', {'class': 'articulo'}).findAll('p')[:-5]:
                texto = texto + paragraph.text
            body.append(texto)
        else:
            body.append('')
            date.append('')
            lead.append('')

    # go to next page
    next_pag_url = base_url + soup.find_all('a', {'class':"lien_pagination"})[i]['href']
    soup = parse_url(next_pag_url)
    if i < 5:
        i += 1
    print('------------------------------------------')

# make dict with lists
data = {
    'date': date,
    'flyer': flyer,
    'lead': lead,
    'headline': headline,
    'body': body,
    'url': url
}

# save everything to a csv file
pd.DataFrame(data).to_csv('data/izq_econ_news.csv')

### Scrapper for "Derecha Diario"
It requires the use of selenium and driver-based navigation

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from time import time, strftime, sleep

In [ ]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from time import time, strftime, sleep

import pandas as pd

url = "https://derechadiario.com.ar/economia"
driver = "./drivers/geckodriver"
delay = 3

options = Options()
options.headless = True
options.page_load_strategy = 'normal'

with webdriver.Firefox(executable_path = driver, options=options) as browser:

    browser.get(url)
    print('url',browser.current_url)
    
    start = time()
    print('Start time:', strftime("%H:%M:%S"))

    count=0

    while True:
        count += 1
        button = browser.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/button')
        if button.text != 'Ver más de Economía':
            print('No more to load')
            break        
        browser.execute_script('arguments[0].scrollIntoView(true);', button)
        sleep(3)

        elements = browser.find_elements(By.TAG_NAME,'article')
        print(f'{count} page(s) loaded with {len(elements)} articles')

        button.click()
        sleep(3)
    
    # save page to parse with bs4
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    news = soup.findAll('a', {'class': 'jsx-1028219248'})
    print(f'{len(news)} articles founded')

    end = time()
    print('End of loading:', strftime("%H:%M:%S"))
    print('Total time:', (end - start)/60)

    browser.quit()

    date = []
    flyer = []
    headline = []
    lead = []
    body = []
    link = []

    # parse and classify
    for i, p_news in enumerate(news):

        url = 'https://derechadiario.com.ar' + p_news['href']
        link.append(url)
        date.append(p_news.findAll('p')[2].text)
        headline.append(p_news.find('h1').text)
        lead.append(p_news.findAll('p')[1].text)
        flyer.append(p_news.findAll('p')[0].text)

        web = requests.get(url)
        article = BeautifulSoup(web.content, 'html.parser')
        content = article.find('div', {'class':"jsx-2701897770 editor body"})
        body.append(content.text)

    data = {
        'date': date,
        'flyer': flyer,
        'title': headline,
        'lead': lead,
        'body': body,
        'url': link
    }

    # save to a csv file
    df = pd.DataFrame(data)
    df.to_csv('derecha_econ_news.csv')